###Mount Drive###

In [1]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [2]:
cd /content/drive/MyDrive/PIE/02/

[Errno 2] No such file or directory: '/content/drive/MyDrive/PIE/02/'
/content


###Imports###

In [2]:
import sys
import time
import cupy as cp
import numpy as np
import matplotlib.pyplot as plt
import cupyx.scipy.fft as cufft
import scipy.fft
from scipy.fft import ifft2, fft2, ifftshift, fftshift
scipy.fft.set_global_backend(cufft)

ModuleNotFoundError: No module named 'cupy'

###Load Diffraction Data###

###Load Position Data###

###Parameter###

In [ ]:
lam = 632.8 * (10**(-9))
pixelnum = 1600
pixelsize = 7.4* (10**(-6))
stepnumber=10
d = 142 * (10**(-3))
k = 2 * np.pi / lam
z1 = 27 * (10**(-3))
z2 = 10 * (10**(-3))
ratio=68

###Coordinate###

In [ ]:
xx = cp.linspace(-pixelsize*pixelnum/2, pixelsize*pixelnum/2, pixelnum)
yy = cp.linspace(-pixelsize*pixelnum/2, pixelsize*pixelnum/2, pixelnum)
[XX, YY] = cp.meshgrid(xx, yy)
fx = cp.linspace(-1/pixelsize/2, 1/pixelsize/2, pixelnum)
fy = cp.linspace(-1/pixelsize/2, 1/pixelsize/2, pixelnum)
[Fx, Fy] = cp.meshgrid(fx, fy)

###Hole###

In [ ]:
hole = (cp.sqrt(XX**2+YY**2)<(1.2*10**(-3))).astype(int)

###Angular Spectrum###

In [ ]:
H1 = cp.exp(1j*k*d*cp.sqrt(1-(lam*Fx)**2-(lam*Fy)**2+0j))
H2 = cp.exp((-1j*k*d*cp.sqrt(1-(lam*Fx)**2-(lam*Fy)**2+0j)).real)
H3 = cp.exp(1j*k*z2*cp.sqrt(1-(lam*Fx)**2-(lam*Fy)**2+0j))

###Propogation and Reconstruction###

In [ ]:
num_iteration = 100
progress_bar = True
Tx = cp.loadtxt('x.txt', unpack = True).T
Ty = cp.loadtxt('y.txt', unpack = True).T

In [ ]:
hole = fftshift(ifft2(ifftshift(H3 * (fftshift(fft2(ifftshift(hole)))))))
r = cp.sqrt(z1**2 + XX**2 + YY**2)
Afield = hole * cp.exp(1j*k*r) * 0.1
Assumption1 = cp.ones((6000,6000)).astype(cp.complex128)

In [ ]:
start_time = time.time()
for i in range(num_iteration):
  for n in range(7):
    for m in range(stepnumber):
      # Print progress
      if(progress_bar == True):
        sys.stdout.write("\rIteration:"+ str(i+1)+ "  Progress:"+ str((10*n+m+1)/0.7) + "%")
        sys.stdout.flush()
      # Get each diffraction image and process it
      filename = str((n)*10+m+1)+".PNG";
      diffraction_image = plt.imread(filename)
      I = 255 * diffraction_image[1031-800:1031+800,1065-800:1065+800]
      I= cp.array(I)
      # Define positional boundary of each diffraction pattern
      x1_assumption1 = 1601 - np.round((Tx[n*stepnumber+m]-Tx[0])/ratio)
      x2_assumption1 = x1_assumption1 + pixelnum
      y1_assumption1 = 2601 - np.round((Ty[n]-Ty[0])/ratio)
      y2_assumption1 = y1_assumption1 + pixelnum

      # Retrieve pixels of reconstructed image within the boundary
      Tem2 = Assumption1[x1_assumption1:x2_assumption1 , y1_assumption1:y2_assumption1]

          # Consider using Ptychography 4.0 Here / Using GPU
      # Reconstruction Process/Calculation
      assump2_step1 = ifftshift(H1 * fftshift(fft2(ifftshift(Afield * Tem2))))
      Assumption2 = fftshift(ifft2(assump2_step1))
      Renew = cp.sqrt(abs(I)) * cp.exp(1j * cp.angle(Assumption2))
      Reconstruction = fftshift(ifft2(ifftshift(fftshift(fft2(ifftshift(Renew))) * cp.conj(H1))))
      change = Reconstruction - Afield * Tem2
      Tem2 = Tem2 + cp.conj(Afield) / (abs(Afield)**2 + 1) * abs(Afield) / abs(Afield).max() * change
      temp=Tem2
      Afield = Afield + cp.conj(temp) / (abs(temp)**2 + 0.1) * abs(temp) / abs(temp).max() * change

      # Update pixels of within the boundary
      Assumption1[x1_assumption1:x2_assumption1 , y1_assumption1:y2_assumption1] = Tem2

  # Plot the reconstucted image per iteration

    fig, (ax1,ax2,ax3) = plt.subplots(1,3)
    plt.gcf().set_size_inches(13, 10)
    plot_1 = ax1.imshow(abs(Assumption1[2000:2600,3000:3600]).get(), cmap='gray')
    plot_2 = ax2.imshow(cp.angle(Assumption1[2000:2600,3000:3600]).get(), cmap='gray')
    plot_3 = ax3.imshow(abs(Afield).get(), cmap='gray')
    plt.show(block=False)
end_time = time.time()

###Final Reconstructed Image###

In [ ]:
fig, (ax1,ax2,ax3) = plt.subplots(1,3)
plt.gcf().set_size_inches(13, 10)
plot_1 = ax1.imshow(abs(Assumption1[2200:3000,2800:3600]).get(), cmap='gray')
plot_2 = ax2.imshow(cp.angle(Assumption1[2200:3000,2800:3600]).get(), cmap='gray')
plot_3 = ax3.imshow(abs(Afield).get(), cmap='gray')
plt.show(block=False)

###Execution Time Comparision###
The speed up will increase rapidly as the number of iteration increases

In [ ]:
ePIE_GPU_time = end_time-start_time
ePIE_time = 515.3737990856171  # got this from ePIE.ipynb
print("e-PIE using GPU Execution Time: "+ str(round(ePIE_GPU_time, 2)) +"s")
print("e-PIE without GPU Execution Time: "+ str(round(ePIE_time, 2)) +"s")
speed_up = ePIE_time / ePIE_GPU_time
print("GPU Speed Up: x" + str(round(speed_up, 2)))